In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from scipy.special import loggamma as lgamma
from scipy.special import comb as Cnk
from scipy.stats import binom, poisson, chi2, gamma

# load the dataset

In [ ]:
hk = np.genfromtxt("../input/covid-offspring/Hong Kong.csv", delimiter=',')[1:]
rwanda = np.genfromtxt("../input/covid-offspring/Rwanda.csv", delimiter=',')[1:]
simulated_data = np.genfromtxt("../input/covid-offspring/Simulated data.csv", delimiter=',')

# Intrinsic Bayes factor

In [ ]:
def BF_intrinsic(X, Y, method='AI'):
    n1 = len(X); n2 = len(Y)
    s1 = sum(X); s2 = sum(Y)
    log_S1 = -(s1 + s2 + 0.5) * np.log(n1 + n2)
    log_S2 = - (s1 + s2) * np.log(n1) - (s2 - 0.5) * np.log(n2/n1) + lgamma(s1 + 0.5) + lgamma(s2 + 0.5) -\
            np.log(n2) - lgamma(s1 + s2 + 1) # analytical integration
############################ numerical underflow ############################
##     S1 = -(s1 + s2 + 0.5) * np.log(n1 + n2)
##     func = lambda x: np.exp((s2 - 0.5) * np.log(x) - (s1 + s2 + 1) * np.log(n1 + n2 * x))
##     S2 = integrate.quad(func, 0, float("inf"))
##     S2 = S2[0]
##     print(S2)
#############################################################################
    
    T_ratio = np.zeros((n1, n2))
    T_ratio_unique_dict = {}
    for i, xi in enumerate(X):
        for j, yj in enumerate(Y):
            try:
                T_ratio[i, j] = T_ratio_unique_dict[(xi, yj)]
            except KeyError:
                func_T_ratio = lambda x: np.exp((yj - 0.5) * np.log(x) - (xi + yj + 0.5) * np.log(1 + x) - 0.5 * np.log(n1 + n2 * x))
                T1 = np.exp((xi + yj + 0.5) * np.log(0.5))
                T2 = integrate.quad(func_T_ratio, 0, np.inf)
                T2 = T2[0]
                T_ratio[i, j] = np.log(T1) - np.log(T2)
                T_ratio_unique_dict[(xi, yj)] = np.log(T1) - np.log(T2)
    
    if method == 'AI':
        BF10 = np.exp(log_S2 - log_S1) * np.mean(np.exp(T_ratio[np.logical_not(np.isnan(T_ratio))]))
    elif method == 'ME':
        BF10 = np.exp(log_S2 - log_S1) * np.exp(np.median(T_ratio[np.logical_not(np.isnan(T_ratio))]))
                
    return BF10      
    

# Fractional Bayes factor

In [ ]:
def BF_fractional(X, Y):
    n1 = len(X); n2 = len(Y)
    s1 = sum(X); s2 = sum(Y)
    m1 = s1 / n1; m2 = s2 / n2
    log_S1 = -(s1 + s2 + 0.5) * np.log(n1 + n2)
    log_S2 = - (s1 + s2) * np.log(n1) - (s2 - 0.5) * np.log(n2/n1) + lgamma(s1 + 0.5) + lgamma(s2 + 0.5) -\
            np.log(n2) - lgamma(s1 + s2 + 1) # analytical integration
    log_S1b = (m1 + m2 + 0.5) * np.log(0.5)
    func = lambda x: np.exp((m2 - 0.5) * np.log(x) - (m1 + m2 + 0.5) * np.log(1 + x) - 0.5 * np.log(n1 + n2 * x))
    S2b = integrate.quad(func, 0, float("inf"))
    log_S2b = np.log(S2b[0])
    
    BF10 = np.exp(log_S2 - log_S1 + log_S1b - log_S2b)
                
    return BF10      
    

# Basic Bayes factor

In [ ]:
def BF_basic(X, Y, a = 1e-3, b = 1e-3, log=False):
    a1 = a; b1 = b; a2 = a; b2 = b
    n1 = len(X); n2 = len(Y)
    s1 = sum(X); s2 = sum(Y)
    
    BF10 = lgamma(s1 + a1) + a1 * np.log(b1) + lgamma(s2 + a2) + a2 * np.log(b2) \
            - lgamma(a1) - (s1 + a1) * np.log(n1 + b1) - lgamma(a2) - (s2 + a2) * np.log(n2 + b2) \
            - lgamma(s1 + s2 + a) - a * np.log(b) + lgamma(a) + (s1 + s2 + a) * np.log(n1 + n2 + b)
    
    if log:
        return BF10
    
    BF10 = np.exp(BF10)
    return BF10


# C-test $p$-value

In [ ]:
def Cp_value(X, Y):
    n1 = len(X); n2 = len(Y)
    s1 = sum(X); s2 = sum(Y)
    p = n1/(n1+n2)
    
    P1 = sum(binom.pmf(np.arange(0, s1 + 1), s1 + s2, p))
    P2 = sum(binom.pmf(np.arange(s1, s1 + s2 + 1), s1 + s2, p))
    
    return 2 * min(P1, P2)


# E-test $p$-value

In [ ]:
def Ep_value(X, Y, cutoff=1e3):
    n1 = len(X); n2 = len(Y)
    s1 = sum(X); s2 = sum(Y)
    mu = (s1 + s2) / (n1 + n2)
    var = s1 / n1 ** 2 + s2 / n2 ** 2
    
    base_T = np.abs(s1 / n1 - s2 / n2) / np.sqrt(var)
    
    xv, yv = np.meshgrid(np.arange(1, cutoff), np.arange(1, cutoff))
    T = np.abs(xv / n1 - yv / n2) / np.sqrt(xv / n1 ** 2 + yv / n2 ** 2)
    log_pmf = poisson.logpmf(xv, n1 * mu) + poisson.logpmf(yv, n2 * mu)
    p = sum(np.exp(log_pmf[T >= base_T]))
    
    return p


# likelihood ratio test $p$-value

In [ ]:
def lkp_value(X, Y):
    XY = np.hstack((X, Y))
    log_lr = sum(poisson.logpmf(XY, XY.mean())) - sum(poisson.logpmf(X, X.mean())) - sum(poisson.logpmf(Y, Y.mean()))
    p_lr = 0.5 * (1 - chi2.cdf(-2 * log_lr, 1))
    
    return p_lr

# posterior predictive $p$-value

In [ ]:
def ppp_value(y_obs, trials=1e3, calib_trials=1e3, calib=2, plotting=False):
    len_y = len(y_obs); sum_y = sum(y_obs)
    a = 0.5; b = 1e-4 # gamma prior
    
    def func_ppp_value(y_obs, trials=1e3, plotting=False):
        mu = gamma.rvs(a + sum_y, scale = 1 / (b + len_y), size = trials)
        y_rep = np.zeros((trials, len_y))
        for i in range(trials):
            y_rep[i] = poisson.rvs(mu[i], size=len_y)
            
        # compute the observed and replicated discrepancy
        T_rep = ((y_rep.T - mu).T ** 2 / mu[:, np.newaxis]).sum(axis=1)
        T_obs = ((y_obs - mu[:, np.newaxis]) ** 2 / mu[:, np.newaxis]).sum(axis=1)
        
        ppp = (T_rep > T_obs).sum() / trials
        
        return ppp
    
    if calib == 2: # ppp without calibration
        return func_ppp_value(y_obs, trials, plotting)
    
    elif calib == 1: # posterior-calibrated ppp
        ppp_obs_new = np.zeros(calib_trials); ppp_obs = np.zeros(calib_trials)
        mu = gamma.rvs(a + sum_y, scale = 1 / (b + len_y), size = calib_trials)
        for i in range(calib_trials):
            y_obs_new = poisson.rvs(mu[i], size=len_y)
            ppp_obs_new[i] = func_ppp_value(y_obs_new, trials)
            ppp_obs[i] = func_ppp_value(y_obs, trials)
            
        ppp_calib = (ppp_obs > ppp_obs_new).sum() / calib_trials
    
        return ppp_calib


# results on simulated data

In [ ]:
np.set_printoptions(4)
for i in range(4):
    x = simulated_data[i, :5]; y = simulated_data[i, 5:]
    print(np.array([Cp_value(x, y), Ep_value(x, y, 1000), lkp_value(x, y), ppp_value(np.hstack((x, y)), trials=int(1e4), calib=2), \
          ppp_value(np.hstack((x, y)), trials=int(1e4), calib_trials=int(1e4), calib=1), \
          BF_intrinsic(x, y, 'AI'), BF_intrinsic(x, y, 'ME'), BF_fractional(x, y), BF_basic(x, y, a=0.5, b=0.001)]))

# results on COVID-19 data

In [ ]:
print(np.array([Cp_value(hk, rwanda), Ep_value(hk, rwanda, 1000), lkp_value(hk, rwanda), ppp_value(np.hstack((hk, rwanda)), trials=1000000, calib=2), \
          ppp_value(np.hstack((hk, rwanda)), trials=1000000, calib_trials=1000000, calib=1), \
          BF_intrinsic(hk, rwanda, 'AI'), BF_intrinsic(hk, rwanda, 'ME'), BF_fractional(hk, rwanda), BF_basic(hk, rwanda, a=0.5, b=0.001)]))